# Solar Irradiance From AC Export

A Jupyter Notebook that does it's best to model and construct a historical solar irradiance time series from solar panel park's historical AC export data.

## 1. Project Setup

### 1.1 Imports

In [ ]:
# --- Imports ---

# Standard Library Imports
from pathlib import Path
import os

# Third-Party Library Imports
import yaml
import pandas as pd
import plotly.io as pio
from dotenv import load_dotenv

print("✅ Libraries loaded successfully.")

***

### 1.2 Configuration

This project uses a two-step configuration process:

1.  **Path Definition (`.env`):** This file defines the project's physical location (`PROJECT_ROOT`) and the name of the configuration file. This separation ensures the notebook is portable across different machines and environments.
2.  **Parameter Definition (`config.yml`):** This file contains the physical and electrical parameters of your solar park(s), including sensitive information like GPS coordinates and detailed system specifications.

**To get started:**

1.  **Configure Paths:** Copy the template file `.env.example` to a new file named `.env`. Open `.env` and set the absolute path for the `PROJECT_ROOT` variable.
2.  **Configure Parks:** Copy the example configuration file `config.example.yml` to `config.yml`. Open `config.yml` and replace the placeholder values with the details of your solar installation.

The cell below loads the environment variables, resolves the final configuration path, and sets up the plotting environment.

In [ ]:
# --- Configuration ---

# Load environment variables from .env file
load_dotenv()

# Define paths using environment variables
PROJECT_ROOT_STR = os.getenv("PROJECT_ROOT")
CONFIG_FILENAME = os.getenv("CONFIG_FILENAME", "config.yml")  # Fallback to config.yml
PRODUCTION_AND_PRICE_FILE_PATH = os.getenv(
    "PRODUCTION_AND_PRICE_FILE_PATH",
    "/home/user/solar-irradiance-from-ac-export/production.csv",
)
WEATHER_FILE_PATH = os.getenv(
    "WEATHER_FILE_PATH", "/home/user/solar-irradiance-from-ac-export/weather.csv"
)
OUTPUT_DIR_STR = os.getenv("OUTPUT_DIR")

if not PROJECT_ROOT_STR:
    # If PROJECT_ROOT is not set in .env, assume the current working directory
    PROJECT_ROOT_STR = os.getcwd()
    print(
        f"⚠️ WARNING: PROJECT_ROOT not set in .env. Using current directory: {PROJECT_ROOT_STR}"
    )

PROJECT_ROOT = Path(PROJECT_ROOT_STR)
CONFIG_PATH = PROJECT_ROOT / CONFIG_FILENAME

if not OUTPUT_DIR_STR:
    # If OUTPUT_DIR is not set, default to a subdirectory within the project root
    OUTPUT_DIR = PROJECT_ROOT / "output"
    print(
        f"⚠️ WARNING: OUTPUT_DIR not set in .env. Using default directory: {OUTPUT_DIR}"
    )
else:
    OUTPUT_DIR = Path(OUTPUT_DIR_STR)

print(f"Project Root defined as: {PROJECT_ROOT}")
print(f"Configuration file path: {CONFIG_PATH}")
print(f"Output directory defined as: {OUTPUT_DIR}")

try:
    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    # Extract park configurations
    PARK_CONFIGS = config.get("parks", {})

    if not PARK_CONFIGS:
        raise ValueError(
            "No parks defined under the 'parks' key in the configuration file."
        )

    # Create a list of park names for easy iteration later
    PARK_NAMES = list(PARK_CONFIGS.keys())

    # --- Load and Validate Target Park for Analysis ---
    TARGET_PARK_NAME = os.getenv("TARGET_PARK_NAME")

    if not TARGET_PARK_NAME:
        raise ValueError("TARGET_PARK_NAME is not set in the .env file. Please specify which park to analyze.")

    if TARGET_PARK_NAME not in PARK_NAMES:
        raise ValueError(
            f"The target park '{TARGET_PARK_NAME}' defined in .env is not found in 'config.yml'.\n"
            f"Available parks in config: {PARK_NAMES}"
        )

    print(f"🎯 Analysis will be performed for target park: '{TARGET_PARK_NAME}'")

    print(
        f"✅ Configuration loaded successfully from '{CONFIG_PATH}' for {len(PARK_NAMES)} park(s): {', '.join(PARK_NAMES)}."
    )

except FileNotFoundError:
    print(f"❌ CONFIGURATION ERROR: The '{CONFIG_PATH}' file was not found.")
    print(
        "Please check your .env file's PROJECT_ROOT setting, and ensure 'config.yml' exists at that location."
    )
    print(
        "If 'config.yml' is missing, copy 'config.example.yml' to 'config.yml' and fill in your park's details."
    )
except (yaml.YAMLError, ValueError) as e:
    print(
        f"❌ CONFIGURATION ERROR: Could not parse '{CONFIG_PATH}'. Please check its format. Details: {e}"
    )


# --- Plotting and Display Configuration ---
pio.templates.default = "plotly_dark"

# Set display options for better viewing in Jupyter
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print("Plotting and display options set.")

***
***

## 2. Data Loading

### Helper Functions

In [ ]:
# --- Data Loading Helper Function ---


def load_park_specific_data(
    file_path: str,
    timestamp_col: str,
    park_name_col: str,
    required_data_cols: list[str],
    target_park_name: str,
    data_name: str,
) -> pd.DataFrame:
    """
    Loads, validates, and filters data for a single specified park from a long-format CSV.

    Args:
        file_path (str): Absolute path to the CSV file.
        timestamp_col (str): Name of the timestamp column.
        park_name_col (str): Name of the park identifier column.
        required_data_cols (list): List of required data column names.
        target_park_name (str): The specific park to extract data for.
        data_name (str): A descriptive name for the data (e.g., "Production").

    Returns:
        pandas.DataFrame: A DataFrame containing only the data for the target park,
                          with the park_name column removed. Returns an empty
                          DataFrame on failure.
    """
    print(f"--- Loading {data_name} Data for '{target_park_name}' ---")
    print(f"Attempting to load from: {file_path}")

    try:
        # 1. Load the full CSV
        df = pd.read_csv(
            file_path, parse_dates=[timestamp_col], index_col=timestamp_col
        )

        # 2. Basic Column Check
        all_required_cols = required_data_cols + [park_name_col]
        if not all(col in df.columns for col in all_required_cols):
            missing = [col for col in all_required_cols if col not in df.columns]
            raise ValueError(f"Missing required columns in {data_name} CSV: {missing}")

        # 3. Data Cleaning and Validation
        df.index = pd.to_datetime(df.index, utc=True)
        df = df.dropna(subset=[park_name_col])
        df[park_name_col] = df[park_name_col].astype(str)

        # 4. Check if Target Park Exists in Data
        if target_park_name not in df[park_name_col].unique():
            raise ValueError(
                f"Target park '{target_park_name}' not found in the {data_name} file."
            )

        # 5. Filter for Target Park and Finalize
        df_park = df[df[park_name_col] == target_park_name].copy()

        # Drop the now-redundant park name column
        df_park = df_park.drop(columns=[park_name_col])

        df_park = df_park.sort_index()
        print(f"✅ {data_name} data for '{target_park_name}' loaded successfully.")
        print(f"   Shape of final DataFrame: {df_park.shape}")
        print(f"   Time range: {df_park.index.min()} to {df_park.index.max()}")
        print("Sample:")
        print(df_park.sample(n=5))
        return df_park

    except FileNotFoundError:
        print(f"❌ DATA ERROR: The {data_name} file was not found at: {file_path}")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ AN UNEXPECTED ERROR OCCURRED during {data_name} data loading: {e}")
        return pd.DataFrame()

print("✅ Helper function load_park_specific_data defined.")

***

### 2.1 Hourly Production And Spot Price Data

In [ ]:
# --- Load Production and Price Data ---

# Define required column names for production data
COL_TIMESTAMP = "timestamp_utc"
COL_PARK_NAME = "park_name"
PRODUCTION_DATA_COLS = ["ac_export_kwh", "spot_price_eur_mwh"]

# Load the data for the target park using the helper function
df_production = load_park_specific_data(
    file_path=PRODUCTION_AND_PRICE_FILE_PATH,
    timestamp_col=COL_TIMESTAMP,
    park_name_col=COL_PARK_NAME,
    required_data_cols=PRODUCTION_DATA_COLS,
    target_park_name=TARGET_PARK_NAME,  # type: ignore
    data_name="Production & Price",
)
assert isinstance(df_production.index, pd.DatetimeIndex)

***

### 2.2 Load Hourly Weather Data

In [ ]:
# --- Load and Crop Weather Data ---

# Define required column names for weather data
WEATHER_DATA_COLS = ["temp_air_c", "wind_speed_m_s", "pressure_hpa", "ghi_w_m2"]

# Load the weather data for the target park using the helper function
df_weather = load_park_specific_data(
    file_path=WEATHER_FILE_PATH,
    timestamp_col=COL_TIMESTAMP,
    park_name_col=COL_PARK_NAME,
    required_data_cols=WEATHER_DATA_COLS,
    target_park_name=TARGET_PARK_NAME,  # type: ignore
    data_name="Weather",
)
assert isinstance(df_weather.index, pd.DatetimeIndex)

# Post-processing: Crop the weather data to the production time range
if not df_production.empty and not df_weather.empty:
    start_time = df_production.index.min()
    end_time = df_production.index.max()

    original_rows = len(df_weather)
    df_weather = df_weather.loc[start_time:end_time].copy()

    print(f"\nWeather data cropped to production time range.")
    print(f"   Original rows: {original_rows}, Cropped rows: {len(df_weather)}")
    print(f"   New time range: {df_weather.index.min()} to {df_weather.index.max()}")

***
***

## 3. Data Upsampling and Feature Engineering 

### Helper Functions 

In [ ]:
# --- Interpolation Helper Function ---

from typing import Literal
import numpy as np


def interpolate_by_gap_size(
    data: pd.Series | pd.DataFrame,
    max_gap_size: int = 1,
    method: Literal[
        "linear",
        "time",
        "index",
        "values",
        "pad",
        "nearest",
        "zero",
        "slinear",
        "quadratic",
        "cubic",
        "barycentric",
        "polynomial",
        "spline",
        "krogh",
        "piecewise_polynomial",
        "pchip",
        "akima",
        "cubicspline",
        "from_derivatives",
    ] = "linear",
    **kwargs
) -> pd.Series | pd.DataFrame:
    """
    Interpolates NaN values in a Series or DataFrame, but only for gaps
    that are less than or equal to a specified maximum size.

    For a DataFrame, the operation is applied column-wise.

    Args:
        data (pd.Series | pd.DataFrame): Input data with potential NaNs.
        max_gap_size (int): Max consecutive NaNs to interpolate. Gaps larger
                            than this are ignored. Defaults to 1.
        method (str): Interpolation technique (see pandas.Series.interpolate).
                      Defaults to "linear".
        **kwargs: Additional keyword arguments for the interpolate() method.

    Returns:
        pd.Series | pd.DataFrame: New object with specified gaps filled.
    """
    if not isinstance(data, (pd.Series, pd.DataFrame)):
        raise TypeError("Input `data` must be a pandas Series or DataFrame.")
    if not isinstance(max_gap_size, int) or max_gap_size <= 0:
        raise ValueError("`max_gap_size` must be a positive integer.")

    if isinstance(data, pd.Series):
        if data.empty or data.isna().sum() == 0:
            return data.copy()

        # Identify gaps by creating groups based on non-NaN values
        grouper = data.notna().cumsum()
        # Calculate the size of each NaN gap
        gap_sizes = data.isna().groupby(grouper).transform("sum")
        # Create a boolean mask for NaNs that are part of small-enough gaps
        mask_to_fill = data.isna() & (gap_sizes <= max_gap_size)

        # Interpolate the entire series to get the fill values
        fully_interpolated = data.interpolate(method=method, **kwargs) # type: ignore

        # Apply the fill values only where the mask is True
        return data.where(~mask_to_fill, fully_interpolated)

    # If it's a DataFrame, apply this function to each column
    return data.apply(
        lambda col: interpolate_by_gap_size(
            col, max_gap_size=max_gap_size, method=method, **kwargs
        )
    )


print("✅ Helper function interpolate_by_gap_size defined.")

***

### 3.1 Upsample Production Data 

In [ ]:
# --- Upsample Production and Price Data to 30-Minute Frequency ---

print(f"--- Upsampling data for park: '{TARGET_PARK_NAME}' ---")

# 1. Determine the actual production period to avoid creating a massive index
first_prod_time: pd.Timestamp = df_production[df_production["ac_export_kwh"] > 0].index.min()
last_prod_time: pd.Timestamp = df_production[df_production["ac_export_kwh"] > 0].index.max()

if pd.isna(first_prod_time) or pd.isna(last_prod_time):
    print("⚠️ No valid production data (> 0 kWh) found. Cannot proceed.")
    df_production_30min = pd.DataFrame()
else:
    print(f"   - Production data range: {first_prod_time} to {last_prod_time}")

    # Crop the hourly data to the actual production period
    df_prod_cropped = df_production.loc[first_prod_time:last_prod_time]

    # Create a full 30-minute time range for this period
    full_30min_range = pd.date_range(
        start=first_prod_time, end=last_prod_time, freq="30min", tz="UTC"
    )

    # 2. Upsample spot price using forward-fill
    # The price at HH:30 is the same as the price at HH:00
    price_30min = (
        df_prod_cropped["spot_price_eur_mwh"]
        .reindex(full_30min_range)
        .ffill()
        .to_frame()
    )

    # 3. Upsample production data from hourly kWh to 30-min average Power (W)
    # Convert kWh (energy) to W (power) for the hourly interval
    power_w_hourly = df_prod_cropped["ac_export_kwh"] * 1000

    # Shift the index 30 mins forward. The power calculated from the interval
    # [HH:00, HH+1:00] is best represented at the midpoint, HH:30.
    power_w_hourly.index = power_w_hourly.index + pd.Timedelta(minutes=30)

    # Reindex to the full 30-min range. This introduces NaNs at HH:00.
    # Use interpolate_by_gap_size(max_gap_size=1) to linearly fill only the
    # points at HH:00, which are surrounded by two HH:30 points.
    power_w_30min = interpolate_by_gap_size(
        power_w_hourly.reindex(full_30min_range), max_gap_size=1, method="linear"
    ).to_frame("ac_export_w") # type: ignore

    # 4. Combine the upsampled series into a single DataFrame
    df_production_30min = price_30min.join(power_w_30min)

    print("\n✅ Production and price data upsampled to 30-minute frequency.")
    print(f"   - Shape of new DataFrame: {df_production_30min.shape}")
    print(
        f"   - New time range: {df_production_30min.index.min()} to {df_production_30min.index.max()}"
    )
    print("   - Sample data:")
    print(df_production_30min.sample(n=5))

***

### 3.2 Upsample Weather Data

In [ ]:
# --- Upsample Weather Data to 30-Minute Frequency ---

print("--- Upsampling weather data ---")

# 1. Use the index from the upsampled production data for perfect alignment
target_30min_range = df_production_30min.index
print(
    f"   - Aligning to target time range: {target_30min_range.min()} to {target_30min_range.max()}"
)

# 2. Handle standard weather variables with linear interpolation
standard_weather_cols = ["temp_air_c", "wind_speed_m_s", "pressure_hpa"]
df_weather_standard_30min = interpolate_by_gap_size(
    df_weather[standard_weather_cols].reindex(target_30min_range),
    max_gap_size=1,
    method="linear",
)

# 3. Handle GHI with a time shift before interpolation
# The hourly GHI value at HH:00 represents the average over the interval [HH-1:00, HH:00].
# We shift the timestamp back 30 mins to place this value at the interval's midpoint (HH-1:30).
ghi_hourly = df_weather[["ghi_w_m2"]].copy()
assert isinstance(ghi_hourly.index, pd.DatetimeIndex)
ghi_hourly.index = ghi_hourly.index - pd.Timedelta(minutes=30)

# Reindex and interpolate. This will correctly fill the values at the top of the hour (HH:00).
df_ghi_30min = interpolate_by_gap_size(
    ghi_hourly.reindex(target_30min_range), max_gap_size=1, method="linear"
)

# 4. Combine all upsampled weather series
df_weather_30min = df_weather_standard_30min.join(df_ghi_30min)

print("\n✅ Weather data upsampled to 30-minute frequency.")
print(f"   - Shape of new DataFrame: {df_weather_30min.shape}")
print("   - Sample data:")
print(df_weather_30min.sample(n=5))

***

### 3.3 Model PVLIB Features

In [ ]:
# --- Model Solar Geometry and Clear-Sky Irradiance with PVLIB ---

import pvlib

print(f"--- Modeling PVLIB features for '{TARGET_PARK_NAME}' ---")

# 1. Extract location parameters from the main config
park_params = PARK_CONFIGS[TARGET_PARK_NAME]
location = pvlib.location.Location(
    latitude=park_params["location"]["latitude"],
    longitude=park_params["location"]["longitude"],
    tz="UTC",
)

# 2. Prepare inputs for pvlib calculations
times = df_weather_30min.index
pressure_pa = df_weather_30min["pressure_hpa"] * 100  # Convert hPa to Pa
temperature_c = df_weather_30min["temp_air_c"]

# 3. Perform pvlib calculations
print("   - Calculating solar position...")
solar_position = location.get_solarposition(
    times, pressure=pressure_pa, temperature=temperature_c  # type: ignore
)
assert isinstance(solar_position, pd.DataFrame)

# Fill any NaN in apparent_zenith by falling back to the geometric zenith.
# This occurs when atmospheric data (pressure, temp) is missing, which is needed
# for refraction correction. This fallback prevents NaN propagation.
solar_position["apparent_zenith"] = solar_position["apparent_zenith"].fillna(
    solar_position["zenith"]
)

print("   - Calculating airmass...")
airmass_relative = pvlib.atmosphere.get_relative_airmass(
    zenith=solar_position["apparent_zenith"]
)
assert isinstance(airmass_relative, pd.Series)
airmass_absolute = pvlib.atmosphere.get_absolute_airmass(
    airmass_relative=airmass_relative, pressure=pressure_pa  # type: ignore
)
assert isinstance(airmass_absolute, pd.Series)

# For timestamps with missing pressure data, airmass_absolute will be NaN.
# In these cases, fall back to using airmass_relative. This is equivalent to
# assuming standard sea-level pressure, a reasonable approximation to avoid
# data loss in the clear-sky model, especially over long data gaps.
airmass_absolute = airmass_absolute.fillna(airmass_relative)

print("   - Calculating extraterrestrial radiation...")
dni_extra = pvlib.irradiance.get_extra_radiation(times)

print("   - Calculating clear-sky irradiance (Ineichen model)...")
clearsky_irrad = location.get_clearsky(
    times,
    model="ineichen",
    solar_position=solar_position,
    airmass_absolute=airmass_absolute,
    dni_extra=dni_extra,
)
assert isinstance(clearsky_irrad, pd.DataFrame)
clearsky_irrad = clearsky_irrad.rename(
    columns={
        "ghi": "ghi_clearsky_w_m2",
        "dni": "dni_clearsky_w_m2",
        "dhi": "dhi_clearsky_w_m2",
    }
)

print("   - Adding 'is_day' flag...")
is_day = solar_position["apparent_zenith"] < 90.0

# 4. Assemble the final pvlib DataFrame
df_pvlib_30min = pd.concat(
    [
        solar_position[["apparent_zenith", "zenith", "azimuth"]],
        pd.Series(airmass_relative, name="airmass_relative"),
        pd.Series(airmass_absolute, name="airmass_absolute"),
        pd.Series(dni_extra, name="dni_extra_w_m2"),
        clearsky_irrad,
        pd.Series(is_day, name="is_day"),
    ],
    axis=1,
)

print("\n✅ PVLIB features modeled successfully.")
print(f"   - Shape of new DataFrame: {df_pvlib_30min.shape}")
print("   - Sample data:")
print(df_pvlib_30min.sample(n=5))

***

### 3.4 Merge DataFrames 

In [ ]:
# --- Merge All 30-Minute DataFrames ---

print("--- Merging all 30-minute data sources ---")

# Join the three dataframes on their common DatetimeIndex
# This preserves all rows and fills with NaNs where data is missing in one source
df_30min = df_production_30min.join([df_weather_30min, df_pvlib_30min])

print("\n✅ All data sources successfully merged into a single DataFrame.")
print(f"   - Final shape: {df_30min.shape}")
print(f"   - Final time range: {df_30min.index.min()} to {df_30min.index.max()}")
print("   - Sample of merged data:")
print(df_30min.sample(n=5))

***
***

## 4. Anomaly Flagging

### 4.1 Flag Clipping and Curtailment

In [ ]:
# --- Flag Clipping and Curtailment Events ---

assert isinstance(df_30min.index, pd.DatetimeIndex)
print(f"--- Flagging anomalies for '{TARGET_PARK_NAME}' ---")

# --- Configuration ---
# Fetch park-specific parameters from the config
park_config = PARK_CONFIGS[TARGET_PARK_NAME]
export_limit_w = park_config["export_limit_kw"] * 1000
pdc0_w = park_config["system"]["pdc0"]

# Define thresholds for flagging logic
# Flag as clipped if power is within this percentage of the export limit
CLIPPING_TOLERANCE_PCT = 0.01
# Flag as curtailed if power is below this percentage of DC capacity during negative prices
CURTAILMENT_POWER_THRESHOLD_PCT = 0.01

clipping_threshold_w = export_limit_w * (1 - CLIPPING_TOLERANCE_PCT)
curtailment_threshold_w = pdc0_w * CURTAILMENT_POWER_THRESHOLD_PCT

print(f"   - Clipping threshold: >= {clipping_threshold_w:.2f} W")
print(
    f"   - Curtailment threshold: < {curtailment_threshold_w:.2f} W (during negative prices)"
)

# Initialize flag columns to False
df_30min["is_clipped"] = False
df_30min["is_curtailed"] = False

# --- Clipping Detection Logic ---
print("\n1. Applying Clipping Detection Logic...")

# Step 1: Directly flag HH:30 points at or near the export limit
is_hh30 = df_30min.index.minute == 30
direct_clip_mask = is_hh30 & (df_30min["ac_export_w"] >= clipping_threshold_w)
df_30min.loc[direct_clip_mask, "is_clipped"] = True
print(f"   - Found {direct_clip_mask.sum()} HH:30 points with direct clipping.")

# Step 2: Propagate flag to adjacent HH:30 points (Temporal Contamination)
clipped_at_hh30 = df_30min["is_clipped"][is_hh30]
neighbor_clip_mask = clipped_at_hh30.shift(
    1, fill_value=False
) | clipped_at_hh30.shift(-1, fill_value=False)

# --- FIX ---
# The original code incorrectly used `neighbor_clip_mask.index`, which flagged all HH:30 points.
# The corrected code uses boolean alignment to flag only the HH:30 points where the neighbor_clip_mask is True.
df_30min.loc[is_hh30, "is_clipped"] |= neighbor_clip_mask
print(
    f"   - Propagated clipping flag to {neighbor_clip_mask.sum()} neighboring HH:30 points."
)

# Step 3: Propagate flag to interpolated HH:00 points (Interpolation Contamination)
is_hh00 = df_30min.index.minute == 0
interpolated_clip_mask = df_30min["is_clipped"].shift(
    1, fill_value=False
) | df_30min["is_clipped"].shift(-1, fill_value=False)
df_30min.loc[is_hh00, "is_clipped"] |= interpolated_clip_mask[is_hh00]
print(f"   - Propagated clipping flag to interpolated HH:00 points.")

# --- Curtailment Detection Logic ---
print("\n2. Applying Curtailment Detection Logic...")

# Step 1: Directly flag HH:30 points with negative prices and low production during the day
# The 'is_day' check prevents flagging nighttime hours where production is naturally zero.
direct_curtail_mask = (
    is_hh30
    & df_30min["is_day"]
    & (df_30min["spot_price_eur_mwh"] < 0)
    & (df_30min["ac_export_w"] < curtailment_threshold_w)
)
df_30min.loc[direct_curtail_mask, "is_curtailed"] = True
print(
    f"   - Found {direct_curtail_mask.sum()} HH:30 points with direct curtailment."
)

# Step 2: Propagate flag to interpolated HH:00 points
interpolated_curtail_mask = df_30min["is_curtailed"].shift(
    1, fill_value=False
) | df_30min["is_curtailed"].shift(-1, fill_value=False)
df_30min.loc[is_hh00, "is_curtailed"] |= interpolated_curtail_mask[is_hh00]
print(f"   - Propagated curtailment flag to interpolated HH:00 points.")

# --- Final Summary ---
total_clipped = df_30min["is_clipped"].sum()
total_curtailed = df_30min["is_curtailed"].sum()
print("\n✅ Anomaly flagging complete.")
print(
    f"   - Total points flagged as clipped: {total_clipped} ({total_clipped / len(df_30min):.2%})"
)
print(
    f"   - Total points flagged as curtailed: {total_curtailed} ({total_curtailed / len(df_30min):.2%})"
)

***

### 4.2 Flag Inter-Row Shading

In [ ]:
# In cell "4.2 Flag Inter-Row Shading"

### 4.2 Flag Inter-Row Shading

# --- Calculate and Flag Inter-Row Shading for Fixed-Tilt Arrays ---
# This step models the geometry of the solar park to identify periods where
# one row of panels casts a shadow on the row behind it. The flagging logic
# is conservative to account for the upsampling process:
# 1. Interval Contamination: The power value at HH:30 represents the energy
#    from the [HH:00, HH+1:00] interval. If shading occurs at either HH:00
#    or HH:30, the entire interval is considered contaminated, and the
#    HH:30 point is flagged.
# 2. Interpolation Contamination: The power value at HH:00 is interpolated
#    from its two HH:30 neighbors. If either neighbor is flagged, the
#    interpolated point is also flagged.

import pvlib

print("Attempting to flag inter-row shading periods with contamination logic...")

# Retrieve the specific configuration for the target park
park_config = PARK_CONFIGS[TARGET_PARK_NAME]
mount_config = park_config.get("mounting")

# Initialize flag column to False before applying logic
df_30min["is_shaded"] = False

# Check if the necessary geometric parameters are defined in config.yml
if mount_config and mount_config.get("gcr"):
    assert isinstance(df_30min.index, pd.DatetimeIndex)
    try:
        gcr = mount_config["gcr"]
        if not (0 < gcr < 1):
            raise ValueError(f"GCR must be between 0 and 1, but got {gcr}")

        # --- Step 0: Calculate the raw, point-in-time shading fraction ---
        shaded_fraction = pvlib.shading.shaded_fraction1d(
            solar_zenith=df_30min["zenith"],
            solar_azimuth=df_30min["azimuth"],
            axis_azimuth=mount_config["axis_azimuth"],
            shaded_row_rotation=mount_config["surface_tilt"],
            collector_width=1.0,
            pitch=1.0 / gcr,
            axis_tilt=0,
            surface_to_axis_offset=0,
            cross_axis_slope=0,
        )

        # Create a temporary mask for when shading geometrically occurs during the day
        point_in_time_shade_mask = (shaded_fraction > 0) & df_30min["is_day"]

        # --- Step 1: Apply Interval Contamination to HH:30 points ---
        print("1. Applying Interval Contamination Logic...")
        is_hh30 = df_30min.index.minute == 30

        # An HH:30 point is shaded if the geometric shading occurs at its own
        # timestamp OR at the start of its interval (the preceding HH:00 point).
        interval_shade_mask = point_in_time_shade_mask | point_in_time_shade_mask.shift(
            1, fill_value=False
        )

        # Apply this logic only to the HH:30 points, which represent the hourly intervals.
        direct_shade_mask = is_hh30 & interval_shade_mask
        df_30min.loc[direct_shade_mask, "is_shaded"] = True
        print(
            f"   - Found {direct_shade_mask.sum()} HH:30 points with direct or interval shading."
        )

        # --- Step 2: Apply Interpolation Contamination to HH:00 points ---
        print("2. Applying Interpolation Contamination Logic...")
        is_hh00 = df_30min.index.minute == 0

        # An HH:00 point is shaded if either of its adjacent HH:30 neighbors is shaded.
        interpolated_shade_mask = df_30min["is_shaded"].shift(
            1, fill_value=False
        ) | df_30min["is_shaded"].shift(-1, fill_value=False)
        df_30min.loc[is_hh00, "is_shaded"] |= interpolated_shade_mask[is_hh00]
        print(f"   - Propagated shading flag to interpolated HH:00 points.")

    except (KeyError, ValueError) as e:
        print(
            f"❌ SHADING ERROR: Could not calculate shading due to missing or invalid config. Details: {e}"
        )
        # The 'is_shaded' column is already False, so no further action needed.
else:
    print(
        "ℹ️ NOTE: Mounting 'gcr' not found in config.yml. Skipping shading calculation."
    )
    # The 'is_shaded' column is already False, so no further action needed.

# --- Final Summary ---
total_shaded = df_30min["is_shaded"].sum()
daytime_points = df_30min["is_day"].sum()
pct_shaded = (total_shaded / daytime_points) * 100 if daytime_points > 0 else 0
print("\n✅ Shading flagging complete.")
print(
    f"   - Total points flagged as shaded: {total_shaded} ({pct_shaded:.2f}% of daytime)."
)

***

### 4.3 Visually Verify Flags 

In [ ]:
# --- Visually Verify Anomaly Flags with Interactive Plots ---

import random
import plotly.graph_objects as go


def create_flag_verification_plot(
    df_day: pd.DataFrame,
    flag_col: str,
    title: str,
    flag_color: str,
    normal_color: str = "royalblue",
) -> go.Figure:
    """Creates an interactive bar plot to visualize a specific anomaly flag for a single day."""

    df_plot = df_day.copy()
    df_plot["color"] = df_plot[flag_col].map({True: flag_color, False: normal_color})

    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=df_plot.index,
            y=df_plot["ac_export_w"],
            marker_color=df_plot["color"],
            customdata=df_plot[flag_col],
            hovertemplate="<b>Time</b>: %{x|%H:%M}<br><b>Power</b>: %{y:.0f} W<br><b>Flagged</b>: %{customdata}<extra></extra>",
            # Set period to 30 minutes (in milliseconds) and align bars to the start of the period
            xperiod=30 * 60 * 1000,
            xperiodalignment="middle",
        )
    )

    fig.update_layout(
        title_text=title,
        xaxis_title="Time [UTC]",
        yaxis_title="AC Export [W]",
        bargap=0.05,
    )

    return fig


assert isinstance(df_30min.index, pd.DatetimeIndex)

# --- Plot 1: Clipping Verification ---
clipped_days = df_30min[df_30min["is_clipped"]].index.normalize().unique()  # type: ignore

if not clipped_days.empty:
    random_clipped_day = random.choice(clipped_days)
    df_plot_clip = df_30min[df_30min.index.date == random_clipped_day.date()]

    fig_clip = create_flag_verification_plot(
        df_day=df_plot_clip,
        flag_col="is_clipped",
        title=f"Clipping Verification for {random_clipped_day.strftime('%Y-%m-%d')}",
        flag_color="crimson",
    )
    fig_clip.show()
else:
    print("ℹ️ No days with clipping were found to plot.")

# --- Plot 2: Curtailment Verification ---
curtailed_days = df_30min[df_30min["is_curtailed"]].index.normalize().unique()  # type: ignore

if not curtailed_days.empty:
    random_curtailed_day = random.choice(curtailed_days)
    df_plot_curtail = df_30min[df_30min.index.date == random_curtailed_day.date()]

    fig_curtail = create_flag_verification_plot(
        df_day=df_plot_curtail,
        flag_col="is_curtailed",
        title=f"Curtailment Verification for {random_curtailed_day.strftime('%Y-%m-%d')}",
        flag_color="crimson",
    )
    fig_curtail.show()
else:
    print("ℹ️ No days with curtailment were found to plot.")

# --- Plot 3: Shading Verification ---
if "is_shaded" in df_30min.columns:
    shaded_days = df_30min[df_30min["is_shaded"]].index.normalize().unique()  # type: ignore

    if not shaded_days.empty:
        random_shaded_day = random.choice(shaded_days)
        df_plot_shade = df_30min[df_30min.index.date == random_shaded_day.date()]

        fig_shade = create_flag_verification_plot(
            df_day=df_plot_shade,
            flag_col="is_shaded",
            title=f"Shading Verification for {random_shaded_day.strftime('%Y-%m-%d')}",
            flag_color="crimson",
        )
        fig_shade.show()
    else:
        print("ℹ️ No days with inter-row shading were found to plot.")

***
***

## 5. AC Export -> Plane of Array (POA) Estimation

### Helper Functions

In [ ]:
# --- Helper Functions ---

# This section is now split into two core functions:
# 1. `calculate_effective_pdc`: Determines the system's DC capacity at any given
#    moment, accounting for initial degradation and user-defined adjustments for
#    known issues or upgrades.
# 2. `estimate_poa_and_temp_cell`: The iterative solver that, given an effective
#    DC capacity, reverses the PV power chain to estimate POA irradiance.

import numpy as np
import pandas as pd
import pvlib


def calculate_effective_pdc(
    pdc0: float,
    commissioning_date: pd.Timestamp,
    degradation_rate: float,
    current_timestamp: pd.Timestamp,
    adjustments: list[dict] | None = None,
) -> float:
    """
    Calculates the effective DC capacity at a given timestamp, accounting for
    annual degradation and any specified capacity adjustments.

    Args:
        pdc0: Nameplate DC power at STC (W).
        commissioning_date: The date the system began operation.
        degradation_rate: Annual degradation rate (e.g., 0.005 for 0.5%).
        current_timestamp: The timestamp for which to calculate the capacity.
        adjustments: A pre-processed and sorted list of dictionaries, each
                     defining a capacity adjustment period.

    Returns:
        The effective DC capacity in Watts, or np.nan if the period is excluded.
    """
    # 1. Calculate base degradation
    time_delta = current_timestamp - commissioning_date
    years_passed = max(0, time_delta.total_seconds() / (365.25 * 24 * 3600))
    pdc_degraded = pdc0 * (1 - degradation_rate) ** years_passed

    if not adjustments:
        return pdc_degraded

    # 2. Find and apply the most recent applicable adjustment
    # The 'adjustments' list is sorted by start_date descending.
    for adj in adjustments:
        if current_timestamp >= adj["start_date"]:
            # This is the most recent adjustment rule. Check if it's active.
            if current_timestamp <= adj["end_date"]:
                adj_type = adj["adjustment_type"]

                if adj_type == "exclude":
                    return np.nan  # Exclude this period from analysis

                value = adj["value"]
                if adj_type == "percentage":
                    pdc_degraded *= 1 + value
                elif adj_type == "absolute_w":
                    pdc_degraded += value
            # Whether the rule was active or expired, it was the most recent one,
            # so we can stop searching.
            break

    return max(0, pdc_degraded)


def estimate_poa_and_temp_cell(
    p_ac: float,
    temp_air: float,
    wind_speed: float,
    pdc_effective: float,
    gamma_pmp: float,
    inverter_efficiency: float,
    dc_derate_factor: float,
    temp_model_params: dict[str, float],
) -> tuple[float, float, float, float]:
    """
    Iteratively estimates POA irradiance and cell temperature from AC power,
    using a pre-calculated effective DC capacity.

    Args:
        p_ac: AC power output in Watts.
        temp_air: Ambient air temperature in Celsius.
        wind_speed: Wind speed in m/s.
        pdc_effective: Effective DC power of the system at the given timestamp (W).
        gamma_pmp: Power temperature coefficient (e.g., -0.004 for -0.4%/°C).
        inverter_efficiency: Nominal inverter efficiency (e.g., 0.985).
        dc_derate_factor: A factor to account for unmodeled DC losses (e.g., 0.97).
        temp_model_params: Parameters for the SAPM cell temperature model.

    Returns:
        A tuple containing:
        - Estimated POA irradiance (W/m^2).
        - Estimated cell temperature (°C).
        - Final temperature difference upon convergence (°C).
        - Number of iterations performed.
    """
    if any(pd.isna(val) for val in [p_ac, temp_air, wind_speed, pdc_effective]):
        return np.nan, np.nan, np.nan, np.nan

    if p_ac <= 0 or pdc_effective <= 0:
        return 0.0, temp_air, 0.0, 0

    # Constants and initial guess
    TEMP_REF = 25.0
    IRRAD_REF = 1000.0
    MAX_ITER = 10
    TOLERANCE = 0.1
    temp_cell_guess = temp_air + 20.0
    p_dc = p_ac / inverter_efficiency

    irrad_estimate = np.nan
    temp_cell_new = np.nan
    temp_diff = np.nan

    for i in range(1, MAX_ITER + 1):
        temp_factor = 1 + gamma_pmp * (temp_cell_guess - TEMP_REF)
        if temp_factor <= 0:
            return 0.0, temp_air, np.nan, i

        irrad_estimate = (
            p_dc / (pdc_effective * dc_derate_factor * temp_factor)
        ) * IRRAD_REF
        irrad_estimate = max(0, irrad_estimate)

        temp_cell_new = pvlib.temperature.sapm_cell(
            poa_global=irrad_estimate,
            temp_air=temp_air,
            wind_speed=wind_speed,
            **temp_model_params,
        )
        temp_diff = abs(temp_cell_new - temp_cell_guess)

        if temp_diff < TOLERANCE:
            return irrad_estimate, temp_cell_new, temp_diff, i

        temp_cell_guess = temp_cell_new

    return irrad_estimate, temp_cell_new, temp_diff, MAX_ITER

***

### 5.1 Main Processing and Data Integration

In [ ]:
# --- Main Processing and Data Integration ---

# --- Retrieve Park-Specific Configuration ---
try:
    park_config = PARK_CONFIGS[TARGET_PARK_NAME]
    system_config = park_config["system"]
    temp_model_config = park_config["temperature_model"]

    pdc0: float = system_config["pdc0"]
    gamma_pmp: float = system_config["gamma_pmp"]
    inverter_efficiency: float = system_config["inverter_efficiency"]
    dc_derate_factor: float = system_config.get("dc_derate_factor", 1.0)
    degradation_rate: float = system_config["degradation_rate"]
    commissioning_date_str: str = system_config["commissioning_date"]
    commissioning_ts: pd.Timestamp = pd.to_datetime(commissioning_date_str, utc=True)

    # --- Load and Pre-process DC Capacity Adjustments ---
    dc_adjustments_raw = park_config.get("dc_capacity_adjustments", [])
    dc_adjustments_processed = []
    if dc_adjustments_raw:
        print(
            f"\n🔧 Found {len(dc_adjustments_raw)} DC capacity adjustment period(s). Pre-processing..."
        )
        for adj in dc_adjustments_raw:
            try:
                adj_type = adj["adjustment_type"]
                processed_adj = {
                    "start_date": pd.to_datetime(adj["start_date"], utc=True),
                    "end_date": (
                        pd.to_datetime(adj.get("end_date"), utc=True)
                        if adj.get("end_date")
                        else pd.Timestamp.max.tz_localize("UTC")
                    ),
                    "adjustment_type": adj_type,
                    # Value is not required for the 'exclude' type
                    "value": float(adj["value"]) if adj_type != "exclude" else None,
                }
                dc_adjustments_processed.append(processed_adj)
            except (KeyError, ValueError) as e:
                print(
                    f"⚠️ WARNING: Skipping invalid DC capacity adjustment due to missing/invalid key: {adj}. Error: {e}"
                )

        # Sort by start_date descending to easily find the latest applicable adjustment
        dc_adjustments_processed.sort(key=lambda x: x["start_date"], reverse=True)
        print("✅ Adjustments processed.")
    else:
        print("\nℹ️ No DC capacity adjustments defined for this park.")

    print(f"\n⚙️ Using parameters for '{TARGET_PARK_NAME}':")
    print(f"  - Commissioning Date: {commissioning_ts.date()}")
    print(f"  - Nameplate DC Power (pdc0): {pdc0 / 1e3:,.1f} kW")
    print(f"  - Annual Degradation: {degradation_rate:.1%}")
    print(f"  - Temp. Coefficient (gamma_pmp): {gamma_pmp * 100:.3f} %/°C")
    print(f"  - Inverter Efficiency: {inverter_efficiency:.1%}")
    print(f"  - DC Derate Factor: {dc_derate_factor:.2%}")

except KeyError as e:
    print(
        f"❌ CONFIGURATION ERROR: Missing key {e} for park '{TARGET_PARK_NAME}' in config.yml."
    )
    raise

# --- Define Temperature Model from Config ---
try:
    model_type: str = temp_model_config["model_type"]
    model_name: str = temp_model_config["model_name"]
    temp_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS[model_type][
        model_name
    ]
    print(f"\n🌡️ Using {model_type.upper()} temperature model: '{model_name}'")
except KeyError:
    print(
        f"❌ CONFIGURATION ERROR: Invalid temperature model '{model_type}/{model_name}' specified in config.yml."  # type: ignore
    )
    print(
        "Please check available models in pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS."
    )
    raise

# --- Prepare DataFrame for Re-running ---
# Drop columns from previous runs to avoid conflicts.
cols_to_drop = [
    "pdc_effective_w",
    "poa_estimated_w_m2",
    "temp_cell_estimated_c",
    "temp_convergence_diff",
    "iterations",
]
df_30min = df_30min.drop(columns=cols_to_drop, errors="ignore")

# --- Run POA Estimation ---
estimation_mask = (
    (df_30min["ac_export_w"] > 0)
    & (~df_30min["is_clipped"])
    & (~df_30min["is_curtailed"])
    & (~df_30min["is_shaded"])
    & (df_30min["is_day"])
)
df_analysis = df_30min.loc[estimation_mask].copy()

print(f"\n🔬 Calculating effective DC capacity for {len(df_analysis):,} data points...")

# 1. Calculate effective DC capacity for each timestamp
df_analysis["pdc_effective_w"] = df_analysis.index.to_series().apply(
    lambda ts: calculate_effective_pdc(
        pdc0=pdc0,
        commissioning_date=commissioning_ts,
        degradation_rate=degradation_rate,
        current_timestamp=ts,
        adjustments=dc_adjustments_processed,
    )
)

# FIXME: This is a temporary hacky solution. By setting ac_export_w to NaN for periods
# where pdc_effective_w is NaN (i.e., excluded periods from config), we are avoiding
# downstream issues, particularly in cell 7.1 (Golden DataFrame Construction).
# A more robust solution should be implemented to handle these excluded periods
# more gracefully throughout the entire notebook.
pdc_is_nan_mask = df_analysis["pdc_effective_w"].isna()
if pdc_is_nan_mask.any():
    print(
        f"ℹ️ Nullifying 'ac_export_w' for {pdc_is_nan_mask.sum()} entries due to excluded DC capacity periods."
    )
    # Update both the main DataFrame and the analysis slice to ensure consistency
    df_30min.loc[df_analysis.index[pdc_is_nan_mask], "ac_export_w"] = float("nan")
    df_analysis.loc[pdc_is_nan_mask, "ac_export_w"] = float("nan")

print("✅ Effective DC capacity calculated.")

print(f"🔬 Running POA estimation...")
# 2. Run the iterative estimation using the calculated effective DC capacity
results = df_analysis.apply(
    lambda row: estimate_poa_and_temp_cell(
        p_ac=row["ac_export_w"],
        temp_air=row["temp_air_c"],
        wind_speed=row["wind_speed_m_s"],
        pdc_effective=row["pdc_effective_w"],
        gamma_pmp=gamma_pmp,
        inverter_efficiency=inverter_efficiency,
        dc_derate_factor=dc_derate_factor,
        temp_model_params=temp_model_parameters,
    ),
    axis=1,
)

if not results.empty:
    results_df = pd.DataFrame(
        results.tolist(),
        index=df_analysis.index,
        columns=[
            "poa_estimated_w_m2",
            "temp_cell_estimated_c",
            "temp_convergence_diff",
            "iterations",
        ],
    )
    # Join both the effective pdc and the estimation results back to the main df
    df_30min = df_30min.join(df_analysis[["pdc_effective_w"]])
    df_30min = df_30min.join(results_df)

print("✅ Estimation complete.")

# --- Final Data Cleanup ---
print("\n🧹 Cleaning up and filling non-estimated periods...")
night_mask = ~df_30min["is_day"]
df_30min.loc[night_mask, "poa_estimated_w_m2"] = 0.0
df_30min.loc[night_mask, "temp_cell_estimated_c"] = df_30min.loc[
    night_mask, "temp_air_c"
]
df_30min["iterations"] = df_30min["iterations"].fillna(0).astype(int)
print(
    "✅ Cleanup complete. Clipped/curtailed daytime periods remain as NaN for irradiance and temperature."
)

# --- Display Results ---
print(
    "\n📊 Sample of dataframe with new estimated POA and effective DC capacity columns:"
)
display(df_30min.sample(5))

***
***

## 6. Modeling GHI from Estimated POA

### 6.1 GHI Modeling

In [ ]:
# --- GHI Modeling ---

import numpy as np
import pandas as pd
import pvlib
from tqdm.notebook import tqdm

print("--- Starting GHI Modeling from Estimated POA ---")

# --- 1. Preparation ---
GHI_MODEL_COLS = [
    "ghi_modeled_w_m2",
    "dni_modeled_w_m2",
    "dhi_modeled_w_m2",
    "kt_modeled",
    "ghi_model_converged",
    "ghi_model_iterations",
]
IRRADIANCE_COLS = GHI_MODEL_COLS[:4]

df_30min.drop(
    columns=[col for col in GHI_MODEL_COLS if col in df_30min.columns],
    inplace=True,
    errors="ignore",
)

df_30min["ghi_model_converged"] = pd.Series(
    pd.NA, index=df_30min.index, dtype="boolean"
)
df_30min["ghi_model_iterations"] = np.nan
for col in IRRADIANCE_COLS:
    df_30min[col] = np.nan

location_params = PARK_CONFIGS[TARGET_PARK_NAME]["location"]
mounting_params = PARK_CONFIGS[TARGET_PARK_NAME]["mounting"]

# --- 2. GHI Reverse Transposition with Progress Bar ---
print("Step 1: Calculating GHI for daytime points with valid POA...")

# A more robust mask: only process daytime points with a positive POA estimate.
# The > 0 condition implicitly handles both NaNs and zeros.
calc_mask = df_30min["is_day"] & (df_30min["poa_estimated_w_m2"] > 0)
df_to_process = df_30min.loc[calc_mask]
assert isinstance(df_to_process.index, pd.DatetimeIndex)

print(f"   - Found {len(df_to_process)} points to process.")

if not df_to_process.empty:
    monthly_groups = df_to_process.groupby(
        [df_to_process.index.year, df_to_process.index.month]
    )
    results_list = []

    pbar = tqdm(monthly_groups, desc="Modeling GHI (monthly chunks)")
    for (year, month), group in pbar:
        pbar.set_postfix_str(f"{year}-{month:02d}")

        ghi_results_chunk = pvlib.irradiance.ghi_from_poa_driesse_2023(
            surface_tilt=mounting_params["surface_tilt"],
            surface_azimuth=mounting_params["surface_azimuth"],
            solar_zenith=group["zenith"],
            solar_azimuth=group["azimuth"],
            poa_global=group["poa_estimated_w_m2"],
            dni_extra=group["dni_extra_w_m2"],
            albedo=location_params["albedo"],
            full_output=True,
        )

        results_list.append(
            pd.DataFrame(
                {
                    "ghi_modeled_w_m2": ghi_results_chunk[0],
                    "ghi_model_converged": ghi_results_chunk[1],
                    "ghi_model_iterations": ghi_results_chunk[2],
                },
                index=group.index,
            )
        )

    if results_list:
        all_results_df = pd.concat(results_list)
        df_30min.update(all_results_df)

# --- 3. GHI Decomposition (ERBS Model) ---
# Decompose only where GHI was successfully modeled and converged.
decomp_mask = df_30min["ghi_modeled_w_m2"].notna() & (
    df_30min["ghi_model_converged"] == True
)
print(f"Step 2: Decomposing GHI for {decomp_mask.sum()} valid points...")

if decomp_mask.any():
    decomposed = pvlib.irradiance.erbs_driesse(
        ghi=df_30min.loc[decomp_mask, "ghi_modeled_w_m2"],
        zenith=df_30min.loc[decomp_mask, "zenith"],
        datetime_or_doy=df_30min.loc[decomp_mask].index,
    ).rename( # type: ignore
        columns={
            "dni": "dni_modeled_w_m2",
            "dhi": "dhi_modeled_w_m2",
            "kt": "kt_modeled",
        }
    )
    df_30min.update(decomposed)

# --- 4. Data Cleaning and Finalization ---
print("Step 3: Cleaning and finalizing modeled irradiance data...")

failed_convergence_mask = df_30min["ghi_model_converged"] == False
df_30min.loc[failed_convergence_mask, IRRADIANCE_COLS] = np.nan
print(
    f"   - Invalidated {failed_convergence_mask.sum()} points due to GHI model non-convergence."
)

unrealistic_dni_mask = df_30min["dni_modeled_w_m2"] > (
    df_30min["dni_clearsky_w_m2"] * 1.05
)
df_30min.loc[unrealistic_dni_mask, IRRADIANCE_COLS] = np.nan
print(
    f"   - Invalidated {unrealistic_dni_mask.sum()} points exceeding the clear-sky DNI limit."
)

night_mask = ~df_30min["is_day"]
df_30min.loc[night_mask, IRRADIANCE_COLS] = df_30min.loc[
    night_mask, IRRADIANCE_COLS
].fillna(0)
df_30min["ghi_model_converged"] = df_30min["ghi_model_converged"].fillna(False)
df_30min.loc[night_mask, "ghi_model_iterations"] = df_30min.loc[
    night_mask, "ghi_model_iterations"
].fillna(0)

for col in IRRADIANCE_COLS:
    if col in df_30min.columns:
        df_30min[col] = df_30min[col].clip(lower=0)

print("\n✅ GHI Modeling and Decomposition complete.")

display_cols = [
    "poa_estimated_w_m2",
    "ghi_modeled_w_m2",
    "dni_modeled_w_m2",
    "dhi_modeled_w_m2",
    "kt_modeled",
    "ghi_clearsky_w_m2",
    "dni_clearsky_w_m2",
    "ghi_model_converged",
    "ghi_model_iterations",
]
display(df_30min[display_cols].sample(5))

***

### 6.2 Visual Verification of GHI Model 

In [ ]:
# --- Visual Verification of GHI Model ---

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

# --- User Configuration ---
# Set to a tuple like (2023, 8) (Year, Month) to select a specific month.
# Set to None to automatically select a random month from the available data.
# selected_month: tuple[int, int] | None = None
selected_month: tuple[int, int] | None = (2022, 6)
# --------------------------

assert isinstance(df_30min.index, pd.DatetimeIndex)

# 1. Determine the target month
if selected_month is None:
    # Get unique (year, month) pairs from the index
    all_months = list(
        df_30min.index.to_series().apply(lambda dt: (dt.year, dt.month)).unique()
    )

    if not all_months:
        raise ValueError("Dataframe is empty or index is invalid.")

    year, month = random.choice(all_months)
    print(f"No month selected, randomly choosing: Year {year}, Month {month}")
else:
    year, month = selected_month
    if not 1 <= month <= 12:
        raise ValueError(f"Invalid month number: {month}. Must be between 1 and 12.")

    # Check if the year is in the available data range
    if df_30min.index.empty:
        raise ValueError("Dataframe is empty.")
    min_year, max_year = df_30min.index.year.min(), df_30min.index.year.max()
    if not min_year <= year <= max_year:
        raise ValueError(
            f"Invalid year: {year}. Data available for years {min_year}-{max_year}."
        )
    print(f"Plotting user-selected month: Year {year}, Month {month}")

# 2. Create a cell-specific dataframe slice for the selected month
# Using partial string indexing is a concise and effective way to slice a DatetimeIndex.
month_str = f"{year}-{month:02d}"
df_month_ghi_verify = df_30min.loc[month_str]

if df_month_ghi_verify.empty:
    raise ValueError(
        f"No data available for the selected month: {month_str}. "
        f"Data range is from {df_30min.index.min()} to {df_30min.index.max()}"
    )

# --- Plotting ---
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("AC Power Export", "GHI: Modeled vs. Reference"),
)

# Row 1: AC Export
fig.add_trace(
    go.Scatter(
        x=df_month_ghi_verify.index,
        y=df_month_ghi_verify["ac_export_w"],
        name="AC Export",
        mode="lines",
    ),
    row=1,
    col=1,
)

# Row 2: GHI Comparison
fig.add_trace(
    go.Scatter(
        x=df_month_ghi_verify.index,
        y=df_month_ghi_verify["ghi_w_m2"],
        name="Reference GHI",
        mode="lines",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=df_month_ghi_verify.index,
        y=df_month_ghi_verify["ghi_modeled_w_m2"],
        name="Modeled GHI",
        mode="lines",
        fill="tonexty",
        fillcolor="rgba(255, 255, 255, 0.15)",
    ),
    row=2,
    col=1,
)

# --- Layout and Final Touches ---
fig.update_layout(
    title_text=f"GHI Model Verification: Month {month}, {year}",
    height=600,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig.update_yaxes(title_text="Power (W)", row=1, col=1)
fig.update_yaxes(title_text="Irradiance (W/m²)", row=2, col=1)
fig.update_xaxes(title_text="Time (UTC)", row=2, col=1)

fig.show()

***

### 6.3 Diagnostic: Daily Energy and Snow Cover Analysis

In [ ]:
# --- Diagnostic: Daily Energy and Snow Cover Analysis ---

import numpy as np
import pandas as pd

# --- 0. Configuration for Snow Flagging ---
# Threshold for mean daily air temperature below which snow cover is considered possible.
POTENTIAL_SNOW_TEMP_THRESHOLD_C = 1.5  # degrees Celsius

# Threshold for the relative energy deficit. If modeled energy is this much lower than
# reference energy (e.g., -70%), it's a strong indicator of an anomaly like snow.
ENERGY_DEFICIT_THRESHOLD_PCT = -70.0  # percent

print("--- Starting Daily Energy and Snow Cover Analysis ---")

# --- 1. Preparation and Synchronization ---
# Select relevant columns for comparison and diagnostics.
df_comp = df_30min[
    [
        "ghi_w_m2",
        "ghi_modeled_w_m2",
        "temp_air_c",
        "is_day",
        "is_clipped",
        "is_curtailed",
        "is_shaded",
    ]
].copy()

# Synchronize NaNs: ensure we only compare timestamps where both series are valid.
# Also, nullify data points flagged for anomalies (clipping, curtailment, shading)
# as they are not suitable for GHI comparison.
anomaly_mask = (
    df_comp["ghi_w_m2"].isna()
    | df_comp["ghi_modeled_w_m2"].isna()
    | df_comp["is_clipped"]
    | df_comp["is_curtailed"]
    | df_comp["is_shaded"]
)
df_comp.loc[anomaly_mask, ["ghi_w_m2", "ghi_modeled_w_m2"]] = np.nan
print(
    "Step 1: Synchronized NaNs and excluded anomalous data points for GHI comparison."
)


# --- 2. Aggregate to Daily Values ---
def daily_energy_wh(series: pd.Series) -> float:
    """Integrates a power series (W) to daily energy (Wh) using the trapezoidal rule."""
    series = series.dropna()
    if len(series) < 2:
        return np.nan
    assert isinstance(series.index, pd.DatetimeIndex)
    time_in_hours = series.index.hour + series.index.minute / 60.0
    return np.trapezoid(y=series.values, x=time_in_hours.values)  # type: ignore


print("Step 2: Aggregating 30-min data to daily summaries...")
# Resample GHI columns to get daily energy
df_daily_analysis = (
    df_comp[["ghi_w_m2", "ghi_modeled_w_m2"]].resample("D").apply(daily_energy_wh)  # type: ignore
)
df_daily_analysis.rename(
    columns={"ghi_w_m2": "ghi_ref_wh_m2", "ghi_modeled_w_m2": "ghi_modeled_wh_m2"},
    inplace=True,
)

# Resample to get mean daily temperature
df_daily_analysis["temp_air_c_mean"] = df_comp["temp_air_c"].resample("D").mean()


# --- 3. Add Daytime Point Count ---
print("Step 3: Counting valid daytime data points per day...")
daytime_points = df_comp.loc[df_comp["is_day"]]
df_daily_analysis["daytime_point_count"] = (
    daytime_points["ghi_w_m2"].resample("D").count()
)


# --- 4. Calculate Deltas ---
print("Step 4: Calculating daily absolute and relative deltas...")
df_daily_analysis["delta_abs_wh_m2"] = (
    df_daily_analysis["ghi_modeled_wh_m2"] - df_daily_analysis["ghi_ref_wh_m2"]
)
df_daily_analysis["delta_rel_pct"] = (
    (df_daily_analysis["delta_abs_wh_m2"] / df_daily_analysis["ghi_ref_wh_m2"]) * 100
).replace([np.inf, -np.inf], np.nan)


# --- 5. Flag Potential Snow Cover (Energy Deficit Method) ---
print("Step 5: Flagging potential snow cover days...")
cond_temp = df_daily_analysis["temp_air_c_mean"] <= POTENTIAL_SNOW_TEMP_THRESHOLD_C

# Condition 1: Significant relative energy deficit.
cond_deficit = df_daily_analysis["delta_rel_pct"] <= ENERGY_DEFICIT_THRESHOLD_PCT

# Condition 2 (Aggressive): Modeled energy is zero, which is a strong indicator of
# complete panel obstruction (e.g., snow) on a cold day, regardless of reference GHI.
cond_zero_model = df_daily_analysis["ghi_modeled_wh_m2"] == 0

# A day is flagged if the temperature is low AND (either there's a large energy deficit
# OR the modeled energy is exactly zero).
# The fillna(False) is crucial for cond_deficit, as NaN relative deltas (e.g., on
# days with zero reference energy) should not satisfy the deficit condition on their own.
df_daily_analysis["is_snow_covered_by_energy"] = cond_temp & (
    cond_deficit.fillna(False) | cond_zero_model
)


# --- 6. Finalize and Display Daily Analysis ---
snow_days_count = df_daily_analysis["is_snow_covered_by_energy"].sum()
print(
    f"\n✅ Daily analysis complete. Flagged {snow_days_count} potential snow cover days."
)

# Display descriptive statistics
print("\nDescriptive Statistics for Daily Analysis:")
display(df_daily_analysis.describe())


# --- 7. Integrate Snow Flags into Main DataFrame ---
print("\n--- Integrating Snow Flags into Main DataFrame ---")

# Get the dates flagged for snow cover
snow_covered_dates = df_daily_analysis[
    df_daily_analysis["is_snow_covered_by_energy"]
].index.normalize() # type: ignore

# Create the flag column by checking if the timestamp's date is in the flagged dates
assert isinstance(df_30min.index, pd.DatetimeIndex)
df_30min["is_snow_covered"] = df_30min.index.normalize().isin(snow_covered_dates)

print(
    "\n✅ Snow cover analysis integrated. 'df_30min' has been updated with the new flag."
)

***

### 6.4 Visualization: Daily Energy and Snow Cover

In [ ]:
# --- Visualization: Daily Energy and Snow Cover ---

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Configuration ---
# Define the date range for the plot.
START_DATE_PLOT = "2022-11-01"
END_DATE_PLOT = "2023-02-28"

# Define a new color scheme for better visual distinction.
COLOR_REF_NORMAL = "skyblue"
COLOR_MODELED_NORMAL = "steelblue"
COLOR_REF_SNOW = "sandybrown"
COLOR_MODELED_SNOW = "orangered"
# Colors for the temperature line based on the threshold
COLOR_TEMP_NORMAL = "lightslategray"
COLOR_TEMP_COLD = "red"

print(f"--- Generating Daily Energy Plot from {START_DATE_PLOT} to {END_DATE_PLOT} ---")

# --- 2. Data Preparation for Plotting ---
# Filter the dataframe for the selected date range
df_plot = df_daily_analysis.loc[START_DATE_PLOT:END_DATE_PLOT].copy()

# Split into normal and snow-flagged days for bar colors
df_plot_normal = df_plot[~df_plot["is_snow_covered_by_energy"]]
df_plot_snow = df_plot[df_plot["is_snow_covered_by_energy"]]

# Create two separate series for temperature based on the threshold for conditional coloring.
# Values that don't meet the condition become NaN, which Plotly uses to create breaks in the line.
df_plot["temp_above_thresh"] = df_plot["temp_air_c_mean"].where(
    df_plot["temp_air_c_mean"] > POTENTIAL_SNOW_TEMP_THRESHOLD_C
)
df_plot["temp_below_thresh"] = df_plot["temp_air_c_mean"].where(
    df_plot["temp_air_c_mean"] <= POTENTIAL_SNOW_TEMP_THRESHOLD_C
)


# --- 3. Create the Figure with a Secondary Y-Axis ---
fig = make_subplots(specs=[[{"secondary_y": True}]])


# --- 4. Add Bar Traces for GHI Energy ---
# Offset groups ensure bars are properly aligned side-by-side.

# Offset Group 0: All "Reference GHI" bars
fig.add_trace(
    go.Bar(
        x=df_plot_normal.index,
        y=df_plot_normal["ghi_ref_wh_m2"],
        name="Reference GHI (Normal)",
        marker_color=COLOR_REF_NORMAL,
        offsetgroup=0,
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(
        x=df_plot_snow.index,
        y=df_plot_snow["ghi_ref_wh_m2"],
        name="Reference GHI (Snow Flag)",
        marker_color=COLOR_REF_SNOW,
        offsetgroup=0,
    ),
    secondary_y=False,
)

# Offset Group 1: All "Modeled GHI" bars
fig.add_trace(
    go.Bar(
        x=df_plot_normal.index,
        y=df_plot_normal["ghi_modeled_wh_m2"],
        name="Modeled GHI (Normal)",
        marker_color=COLOR_MODELED_NORMAL,
        offsetgroup=1,
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Bar(
        x=df_plot_snow.index,
        y=df_plot_snow["ghi_modeled_wh_m2"],
        name="Modeled GHI (Snow Flag)",
        marker_color=COLOR_MODELED_SNOW,
        offsetgroup=1,
    ),
    secondary_y=False,
)

# --- 5. Add Conditionally Colored Line Trace for Temperature ---
# Plot two separate scatter traces for the temperature. Plotly will not connect
# the gaps, effectively creating a single line that changes color.
fig.add_trace(
    go.Scatter(
        x=df_plot.index,
        y=df_plot["temp_above_thresh"],
        name="Mean Air Temp (°C)",
        mode="lines+markers",
        line=dict(color=COLOR_TEMP_NORMAL, dash="dash"),
        marker=dict(size=4),
        legendgroup="temp",  # Group both traces under one legend item
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df_plot.index,
        y=df_plot["temp_below_thresh"],
        name="Mean Air Temp (°C)",
        mode="lines+markers",
        line=dict(color=COLOR_TEMP_COLD, dash="dash"),
        marker=dict(size=4),
        legendgroup="temp",
        showlegend=False,  # Hide the second trace from the legend
    ),
    secondary_y=True,
)


# --- 6. Update Layout and Finalize ---
subtitle = f"Temperature line colored red when at/below the {POTENTIAL_SNOW_TEMP_THRESHOLD_C}°C threshold for potential snow"
fig.update_layout(
    barmode="group",
    title_text=f"Daily GHI & Air Temperature ({START_DATE_PLOT} to {END_DATE_PLOT})<br><sup>{subtitle}</sup>",
    xaxis_title="Date",
    height=600,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    hovermode="x",
)

# Configure y-axes titles
fig.update_yaxes(title_text="Daily Energy (Wh/m²)", secondary_y=False)
fig.update_yaxes(title_text="Mean Air Temperature (°C)", secondary_y=True)


fig.show()

***
***

## 7. Advanced GHI Model Validation 

### 7.1 Golden DataFrame Construction

In [ ]:
# --- Golden DataFrame Construction ---

# --- Imports and Constants ---
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

# --- Configuration for Golden Day Selection ---

# Defines the percentage of daytime data to trim from the edges (morning/evening)
# before fitting the parabola. This helps to focus the fit on the peak portion
# of the day, where the parabolic approximation is more accurate.
# e.g., 0.20 means trimming 10% from the start and 10% from the end of the day.
DAY_EDGE_TRIM_PERCENTAGE = 0.40

# Defines the tolerance for how far the parabola's vertex can be from solar noon.
# A value of 60 minutes is a reasonable starting point.
VERTEX_NOON_TOLERANCE_MINUTES = 60

# Defines the quantile for R² scores to be considered "golden".
R2_QUANTILE_THRESHOLD = 0.90

# Defines the minimum number of valid data points required within a day
# to attempt a parabolic fit. This prevents fitting on sparse data.
# A value of 6 corresponds to at least 3 hours of valid data.
MIN_DATAPOINTS_FOR_FIT = 6


# --- Helper Function ---
def get_parabolic_fit_details(series: pd.Series) -> dict[str, float | int | None]:
    """
    Fits a parabola to a given time series and returns the fit details.

    Args:
        series: A pandas Series with a DatetimeIndex and numeric values.

    Returns:
        A dictionary containing the R-squared value, polynomial coefficients (a, b, c),
        the vertex time in seconds from the start, and the number of data points used.
        Returns None for values if fitting is not possible.
    """
    series_to_fit = series.dropna()
    data_points = len(series_to_fit)

    # A parabola requires at least 3 points to be uniquely defined.
    if data_points < 3:
        return {
            "r2": None,
            "a": None,
            "b": None,
            "c": None,
            "vertex_x_seconds": None,
            "data_points": data_points,
        }

    time_seconds = (series_to_fit.index - series_to_fit.index.min()).total_seconds()
    values = series_to_fit.to_numpy()

    try:
        # Fit a 2nd degree polynomial (parabola)
        coeffs = np.polyfit(time_seconds, values, 2)
        a, b, c = coeffs

        values_predicted = np.polyval(coeffs, time_seconds)
        r2 = r2_score(values, values_predicted)

        # Vertex formula for y = ax^2 + bx + c is x = -b / 2a
        # If 'a' is extremely close to zero, the fit is essentially linear,
        # and the vertex is at a nonsensical, far-off time. We treat this as an invalid vertex.
        if abs(a) < 1e-9:
            vertex_x_seconds = np.nan
        else:
            vertex_x_seconds = -b / (2 * a)

    except (np.linalg.LinAlgError, ValueError):
        # Handle cases where fitting fails
        return {
            "r2": None,
            "a": None,
            "b": None,
            "c": None,
            "vertex_x_seconds": None,
            "data_points": data_points,
        }

    return {
        "r2": r2,
        "a": a,
        "b": b,
        "c": c,
        "vertex_x_seconds": vertex_x_seconds,
        "data_points": data_points,
    }


# --- Main Analysis ---

# --- Step 1: Prepare and Pre-filter the Analysis DataFrame ---
print("Step 1: Preparing and pre-filtering the main analysis DataFrame...")
df_30min_analysis = df_30min.copy()
assert isinstance(df_30min_analysis.index, pd.DatetimeIndex)
df_30min_analysis["date_only"] = df_30min_analysis.index.normalize()

# --- Step 1a: Flag daytime edges for trimming ---
print("   - Flagging daytime edges to improve parabolic fit...")
df_daylight = df_30min_analysis.loc[df_30min_analysis["is_day"]].copy()
df_daylight["day_rank"] = df_daylight.groupby("date_only").cumcount()
df_daylight["day_size"] = df_daylight.groupby("date_only")["is_day"].transform("size")
n_trim = np.floor(df_daylight["day_size"] * DAY_EDGE_TRIM_PERCENTAGE / 2).astype(int)
is_day_edge = (df_daylight["day_rank"] < n_trim) | (
    df_daylight["day_rank"] >= df_daylight["day_size"] - n_trim
)
df_30min_analysis["is_day_edge"] = is_day_edge.reindex(
    df_30min_analysis.index, fill_value=False
)
print(
    f"   - Flagged {df_30min_analysis['is_day_edge'].sum()} data points as daytime edges."
)

# --- Step 1b: Invalidate data points for the entire analysis ---
print(
    "   - Invalidating data points (nighttime, edges, anomalies) by setting to NaN..."
)
# Invalidate nighttime values for both AC and GHI
night_mask = ~df_30min_analysis["is_day"]
df_30min_analysis.loc[night_mask, ["ac_export_w", "ghi_w_m2", "ghi_modeled_w_m2"]] = (
    np.nan
)

# Define and apply invalidation masks for AC power and GHI
ac_invalidation_mask = (
    df_30min_analysis["is_clipped"]
    | df_30min_analysis["is_curtailed"]
    | df_30min_analysis["is_shaded"]
    | df_30min_analysis["is_snow_covered"]
    | (df_30min_analysis["ac_export_w"] <= 0)
    | df_30min_analysis["is_day_edge"]
)
ghi_invalidation_mask = df_30min_analysis["is_day_edge"]

df_30min_analysis.loc[ac_invalidation_mask, "ac_export_w"] = np.nan
# Modeled GHI comes from the AC so it's logical to invalidate that one too to keep things synced.
df_30min_analysis.loc[ac_invalidation_mask, "ghi_modeled_w_m2"] = np.nan
df_30min_analysis.loc[ghi_invalidation_mask, "ghi_w_m2"] = np.nan
print(f"   - Invalidated {ac_invalidation_mask.sum()} AC export data points.")
print(
    f"   - Invalidated {ghi_invalidation_mask.sum()} GHI data points (on top of nighttime)."
)

# --- Step 2: Create daily summary DataFrame from pre-filtered data ---
print("\nStep 2: Creating daily summary DataFrame with parabolic fit details...")
daily_groups = df_30min_analysis.groupby("date_only")

ac_fit_series = daily_groups["ac_export_w"].agg(get_parabolic_fit_details)
ghi_fit_series = daily_groups["ghi_w_m2"].agg(get_parabolic_fit_details)

ac_fit_details = pd.DataFrame(
    ac_fit_series.tolist(), index=ac_fit_series.index
).add_prefix("ac_")
ghi_fit_details = pd.DataFrame(
    ghi_fit_series.tolist(), index=ghi_fit_series.index
).add_prefix("ghi_")

df_daily_analysis = pd.concat([ac_fit_details, ghi_fit_details], axis=1)
df_daily_analysis.index.name = "date"

df_daily_analysis["solar_noon"] = daily_groups["zenith"].idxmin()
# Correctly determine the start time used for fitting for each series.
# This is the timestamp of the first non-NaN value in each day's group,
# which matches the reference used inside the get_parabolic_fit_details function.
df_daily_analysis["ac_fit_start_ts"] = daily_groups["ac_export_w"].apply(
    lambda s: s.first_valid_index()
)
df_daily_analysis["ghi_fit_start_ts"] = daily_groups["ghi_w_m2"].apply(
    lambda s: s.first_valid_index()
)

# --- Step 3: Identify and flag "Golden Days" ---
print("\nStep 3: Identifying and flagging 'Golden Days' based on fit quality...")
df_daily_analysis["ac_vertex_time"] = df_daily_analysis[
    "ac_fit_start_ts"
] + pd.to_timedelta(df_daily_analysis["ac_vertex_x_seconds"], unit="s")
df_daily_analysis["ghi_vertex_time"] = df_daily_analysis[
    "ghi_fit_start_ts"
] + pd.to_timedelta(df_daily_analysis["ghi_vertex_x_seconds"], unit="s")
df_daily_analysis["ac_vertex_noon_diff_min"] = abs(
    (
        df_daily_analysis["ac_vertex_time"] - df_daily_analysis["solar_noon"]
    ).dt.total_seconds()
    / 60
)
df_daily_analysis["ghi_vertex_noon_diff_min"] = abs(
    (
        df_daily_analysis["ghi_vertex_time"] - df_daily_analysis["solar_noon"]
    ).dt.total_seconds()
    / 60
)

r2_ac_threshold = df_daily_analysis["ac_r2"].quantile(R2_QUANTILE_THRESHOLD)
r2_ghi_threshold = df_daily_analysis["ghi_r2"].quantile(R2_QUANTILE_THRESHOLD)
top_percent = int(np.round((1 - R2_QUANTILE_THRESHOLD) * 100))
print(
    f"   - Using AC Power R² threshold: {r2_ac_threshold:.4f} (Top {top_percent}% of fits)"
)
print(
    f"   - Using Reference GHI R² threshold: {r2_ghi_threshold:.4f} (Top {top_percent}% of fits)"
)

is_valid_ac_fit = (
    (df_daily_analysis["ac_data_points"] >= MIN_DATAPOINTS_FOR_FIT)
    & (df_daily_analysis["ac_a"] < 0)
    & (df_daily_analysis["ac_vertex_noon_diff_min"] <= VERTEX_NOON_TOLERANCE_MINUTES)
    & (df_daily_analysis["ac_r2"] > r2_ac_threshold)
)
is_valid_ghi_fit = (
    (df_daily_analysis["ghi_data_points"] >= MIN_DATAPOINTS_FOR_FIT)
    & (df_daily_analysis["ghi_a"] < 0)
    & (df_daily_analysis["ghi_vertex_noon_diff_min"] <= VERTEX_NOON_TOLERANCE_MINUTES)
    & (df_daily_analysis["ghi_r2"] > r2_ghi_threshold)
)
df_daily_analysis["is_golden"] = is_valid_ac_fit & is_valid_ghi_fit

# --- Step 4: Merge daily analysis back and calculate parabolic curves ---
print("\nStep 4: Merging daily stats and calculating parabolic curves...")
columns_to_merge = [
    "is_golden",
    "ac_fit_start_ts",
    "ghi_fit_start_ts",
    "ac_a",
    "ac_b",
    "ac_c",
    "ghi_a",
    "ghi_b",
    "ghi_c",
]
df_30min_analysis = df_30min_analysis.merge(
    df_daily_analysis[columns_to_merge],
    left_on="date_only",
    right_index=True,
    how="left",
)

# Calculate time deltas relative to the correct start time for each series
time_seconds_ac = (
    df_30min_analysis.index - df_30min_analysis["ac_fit_start_ts"]
).dt.total_seconds()  # type: ignore
time_seconds_ghi = (
    df_30min_analysis.index - df_30min_analysis["ghi_fit_start_ts"]
).dt.total_seconds()  # type: ignore

ac_valid_fit_mask = df_30min_analysis["ac_a"].notna() & df_30min_analysis["is_day"]
ghi_valid_fit_mask = df_30min_analysis["ghi_a"].notna() & df_30min_analysis["is_day"]

ac_coeffs = (
    df_30min_analysis.loc[ac_valid_fit_mask, ["ac_a", "ac_b", "ac_c"]].to_numpy().T
)
ghi_coeffs = (
    df_30min_analysis.loc[ghi_valid_fit_mask, ["ghi_a", "ghi_b", "ghi_c"]].to_numpy().T
)

df_30min_analysis["ac_export_parabolic_fit_w"] = np.nan
df_30min_analysis.loc[ac_valid_fit_mask, "ac_export_parabolic_fit_w"] = np.polyval(
    ac_coeffs, time_seconds_ac[ac_valid_fit_mask]
)
df_30min_analysis["ghi_parabolic_fit_w_m2"] = np.nan
df_30min_analysis.loc[ghi_valid_fit_mask, "ghi_parabolic_fit_w_m2"] = np.polyval(
    ghi_coeffs, time_seconds_ghi[ghi_valid_fit_mask]
)

# --- Masking: Set fitted curve values to NaN where the original data was trimmed ---
# The fitted curves should only exist where the underlying data was used for fitting.
mask_edges_or_night = df_30min_analysis["is_day_edge"] | (~df_30min_analysis["is_day"])

df_30min_analysis.loc[
    mask_edges_or_night, ["ac_export_parabolic_fit_w", "ghi_parabolic_fit_w_m2"]
] = np.nan

df_30min_analysis["is_contaminated"] = (
    df_30min_analysis["is_clipped"]
    | df_30min_analysis["is_curtailed"]
    | df_30min_analysis["is_shaded"]
    | df_30min_analysis["is_snow_covered"]
)

# --- Step 5: Finalize and clean DataFrames ---
print("\nStep 5: Finalizing DataFrames for analysis...")
df_daily_analysis.drop(columns=["ac_fit_start_ts", "ghi_fit_start_ts"], inplace=True)

final_columns = [
    "ac_export_w",
    "ghi_w_m2",
    "ghi_modeled_w_m2",
    "ac_export_parabolic_fit_w",
    "ghi_parabolic_fit_w_m2",
    "temp_cell_estimated_c",
    "is_day",
    "is_clipped",
    "is_curtailed",
    "is_shaded",
    "is_day_edge",
    "is_contaminated",
    "is_snow_covered",
    "is_golden",
]
df_30min_analysis = df_30min_analysis[final_columns]

# Fix for FutureWarning: Explicitly convert to nullable boolean dtype before filling.
# This ensures that NaN values are handled correctly before converting to non-nullable bool.
df_30min_analysis["is_golden"] = (
    df_30min_analysis["is_golden"].astype("boolean").fillna(False).astype(bool)
)

print(
    f"\nConstructed daily summary 'df_daily_analysis' with {df_daily_analysis['is_golden'].sum()} golden days."
)
print("Sample of the final Daily DataFrame:")
print(df_daily_analysis.sample(n=5))
print(
    f"\nConstructed final 30-minute analysis 'df_30min_analysis' with shape {df_30min_analysis.shape}."
)
print("Sample of the final 30-minute DataFrame:")
print(df_30min_analysis.sample(n=5))
print("\nDataFrames are now ready for advanced validation and visualization.\n")

***

### 7.2 Verify The Fitting Visually

In [ ]:
# --- Imports for Visualization ---
import calendar
import random
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots

# --- User Configuration ---
# Set to a specific year and month for focused analysis, or leave as None for a random selection.
YEAR_TO_ANALYZE: int | None = 2022
MONTH_TO_ANALYZE: int | None = 6

# --- Data Preparation ---
assert isinstance(df_daily_analysis.index, pd.DatetimeIndex)
if YEAR_TO_ANALYZE is None or MONTH_TO_ANALYZE is None:
    # Select a random month from the available data if not specified
    available_periods = df_daily_analysis.index.to_period("M").unique()
    if not available_periods.empty:
        random_period = random.choice(available_periods)
        year = random_period.year
        month = random_period.month
        print(
            f"No year/month specified. Randomly selected: {calendar.month_name[month]} {year}"
        )
    else:
        year, month = df_daily_analysis.index.year.min(), 1  # Fallback
        print("Warning: No data available to make a random selection.")
else:
    year, month = YEAR_TO_ANALYZE, MONTH_TO_ANALYZE
    print(f"Displaying data for: {calendar.month_name[month]} {year}")

# Filter both DataFrames to the selected month
month_mask_daily = (df_daily_analysis.index.year == year) & (
    df_daily_analysis.index.month == month
)
df_daily_month = df_daily_analysis[month_mask_daily]

assert isinstance(df_30min_analysis.index, pd.DatetimeIndex)
month_mask_30min = (df_30min_analysis.index.year == year) & (
    df_30min_analysis.index.month == month
)
df_30min_month = df_30min_analysis[month_mask_30min]


# --- Visualization ---
if df_30min_month.empty:
    print("\nNo 30-minute data available for the selected period.")
else:
    fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.4, 0.4, 0.2],
    )

    # --- Row 1: Irradiance ---
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["ghi_w_m2"],
            mode="lines",
            name="GHI (Reference)",
            line={"color": "skyblue"},
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["ghi_parabolic_fit_w_m2"],
            mode="lines",
            name="GHI (Fit)",
            line={"dash": "dash", "color": "cyan"},
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["ghi_modeled_w_m2"],
            mode="lines",
            name="GHI (Modeled)",
            line={"color": "orange"},
        ),
        row=1,
        col=1,
    )

    # --- Row 2: AC Power ---
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["ac_export_w"],
            mode="lines",
            name="AC Power",
            line={"color": "lightgreen"},
        ),
        row=2,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["ac_export_parabolic_fit_w"],
            mode="lines",
            name="AC Power (Fit)",
            line={"dash": "dash", "color": "lime"},
        ),
        row=2,
        col=1,
    )

    # --- Row 3: Cell Temperature ---
    fig.add_trace(
        go.Scatter(
            x=df_30min_month.index,
            y=df_30min_month["temp_cell_estimated_c"],
            mode="lines",
            name="Cell Temp.",
            line={"color": "red"},
        ),
        row=3,
        col=1,
    )

    # --- Add vertical lines for golden days ---
    # FIXME: add_vline has a known bug where the x coord needs to be defined in a hacky way
    # golden_days_in_month = df_daily_month[df_daily_month["is_golden"]]
    # for ts in golden_days_in_month["solar_noon"]:
    #     fig.add_vline(
    #         x=ts,
    #         line_width=1,
    #         line_dash="dash",
    #         line_color="gold",
    #         annotation_text="Golden Day",
    #         annotation_position="top left",
    #     )

    # --- Layout ---
    fig.update_layout(
        title_text=f"Monthly GHI and AC Power Analysis for {calendar.month_name[month]} {year}",
        height=800,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    fig.update_yaxes(title_text="Irradiance (W/m²)", row=1, col=1)
    fig.update_yaxes(title_text="AC Power (W)", row=2, col=1)
    fig.update_yaxes(title_text="Cell Temp (°C)", row=3, col=1)

    fig.show()

# --- Daily Summary Display ---
print(f"\nDaily Summary for {calendar.month_name[month]} {year}:")
display(df_daily_month)

***

### 7.3 Visual Comparison on Golden Days 

In [ ]:
# --- Visual Comparison on Golden Days ---

import plotly.graph_objects as go
import pandas as pd
import numpy as np

if df_30min_analysis.empty:
    print("The 'golden' dataset is empty. Cannot generate the time series plot.")
else:
    # Create a copy and mask non-golden days with NaN to ensure Plotly renders gaps.
    df_golden_only = df_30min_analysis.copy()

    # Mask to select data points that are NOT on a golden day OR are contaminated
    # (even if on a golden day, contamination should be excluded from validation view)
    non_golden_mask = (
        ~df_golden_only["is_golden"]
        | df_golden_only["is_contaminated"]
        | ~df_golden_only["is_day"]
    )

    # Apply the mask to the GHI columns
    df_golden_only.loc[non_golden_mask, ["ghi_modeled_w_m2", "ghi_w_m2"]] = np.nan

    # Reindex for gapped plotting (this ensures all time points exist, even if NaN)
    full_range_index = pd.date_range(
        start=df_golden_only.index.min(),
        end=df_golden_only.index.max(),
        freq="30min",
    )
    df_plot = df_golden_only.reindex(full_range_index)

    # --- Create the plot using graph_objects ---
    fig = go.Figure()

    # Add Modeled GHI trace
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot["ghi_modeled_w_m2"],
            name="Modeled GHI (AC-derived)",
            mode="lines",
            line=dict(color="#1f77b4"),
        )
    )

    # Add Reference GHI trace
    fig.add_trace(
        go.Scatter(
            x=df_plot.index,
            y=df_plot["ghi_w_m2"],
            name="Reference GHI (Satellite or Station)",
            mode="lines",
            line=dict(color="#2ca02c"),
        )
    )

    # --- Update layout and axis titles ---
    fig.update_layout(
        title=f"Modeled vs. Reference GHI on Clear-Sky Golden Days for {TARGET_PARK_NAME}",
        legend_title_text="GHI Source",
        yaxis_title="GHI (W/m²)",
        xaxis_title="Date / Time (UTC)",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    fig.show()

***

### 7.4 Regression Analysis

#### Helper Functions

In [ ]:
# --- Helper Functions ---

import plotly.graph_objects as go
import plotly.colors as pcolors
import statsmodels.api as sm
import colorsys
from statsmodels.regression.linear_model import RegressionResultsWrapper


def plot_regression_scatter(
    df_reg: pd.DataFrame,
    x_col: str,
    y_col: str,
    ols_model: RegressionResultsWrapper,
    title: str,
    x_label: str,
    y_label: str,
) -> go.Figure:
    """
    Generates a standardized regression scatter plot with a Year-Quarter color scheme.
    """
    # --- Color Preparation ---
    df_plot = df_reg.copy()
    assert isinstance(df_plot.index, pd.DatetimeIndex)
    df_plot["Timestamp"] = df_plot.index
    df_plot["Year"] = df_plot.index.year
    df_plot["Quarter"] = df_plot.index.quarter

    unique_years = sorted(df_plot["Year"].unique())
    base_colors_hex = pcolors.qualitative.Plotly
    year_colors = {
        year: base_colors_hex[i % len(base_colors_hex)]
        for i, year in enumerate(unique_years)
    }
    quarter_lightness = {1: 0.80, 2: 0.65, 3: 0.50, 4: 0.35}

    color_map = {}
    for year, hex_color in year_colors.items():
        color_map[year] = {}
        rgb_float = tuple(c / 255.0 for c in pcolors.hex_to_rgb(hex_color))
        h, l, s = colorsys.rgb_to_hls(*rgb_float)
        for quarter, lightness_mod in quarter_lightness.items():
            new_rgb_float = colorsys.hls_to_rgb(h, lightness_mod, s)
            new_rgb_int = tuple(int(c * 255) for c in new_rgb_float)
            color_map[year][quarter] = f"rgb{new_rgb_int}"

    # --- Plotting ---
    fig = go.Figure()

    # Add scatter traces
    for (year, quarter), group in df_plot.groupby(["Year", "Quarter"]):
        fig.add_trace(
            go.Scatter(
                x=group[x_col],
                y=group[y_col],
                mode="markers",
                marker=dict(color=color_map[year][quarter], opacity=0.6, size=5),
                name=f"{year}-Q{quarter}",
                customdata=group[["ghi_modeled_w_m2", "ghi_w_m2", "Timestamp"]],
                hovertemplate=(
                    "<b>%{customdata[2]|%Y-%m-%d %H:%M}</b><br><br>"
                    f"{x_label}: %{{x:.1f}}<br>"
                    f"{y_label}: %{{y:.2f}}<br>"
                    "Modeled GHI: %{customdata[0]:.1f}<br>"
                    "Reference GHI: %{customdata[1]:.1f}"
                    "<extra></extra>"
                ),
            )
        )

    # Add OLS trendline
    x_range = [df_plot[x_col].min(), df_plot[x_col].max()]
    y_range = [ols_model.params["const"] + ols_model.params[x_col] * x for x in x_range]
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_range,
            mode="lines",
            line=dict(color="red", width=2),
            name="OLS Trendline",
        )
    )

    # --- Layout ---
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        legend_title="Year-Quarter",
    )

    return fig

print("Helper function plot_regression_scatter defined.")

***

#### 7.4.1 Regression for Absolute Delta

Here we analyze the absolute error `(Modeled - Reference)` in W/m² to identify any temperature-dependent additive bias in the model. 

In [ ]:
# --- Regression for Absolute Delta ---

# --- 0. Configuration ---
# Define the minimum estimated cell temperature (°C) required for data points
# to be included in the regression analysis. This helps filter out cold
# periods (potential snow cover) that skew results.
MIN_CELL_TEMP_FOR_REGRESSION_C = 0.0

# --- 1. Data Preparation ---
# Filter data to exclude low cell temperatures
df_abs_reg = df_golden_only[
    df_golden_only["temp_cell_estimated_c"] >= MIN_CELL_TEMP_FOR_REGRESSION_C
].copy()
df_abs_reg["delta_ghi_w_m2"] = df_abs_reg["ghi_modeled_w_m2"] - df_abs_reg["ghi_w_m2"]

# --- 2. OLS Regression ---
X = sm.add_constant(df_abs_reg["temp_cell_estimated_c"])
y = df_abs_reg["delta_ghi_w_m2"]
ols_model_abs = sm.OLS(y, X, missing="drop").fit()

# --- 3. Plotting ---
fig_abs = plot_regression_scatter(
    df_reg=df_abs_reg,
    x_col="temp_cell_estimated_c",
    y_col="delta_ghi_w_m2",
    ols_model=ols_model_abs,
    title=(
        f"Absolute Model Error vs. Cell Temperature for {TARGET_PARK_NAME}\n"
        f"(Filtered: Cell Temp >= {MIN_CELL_TEMP_FOR_REGRESSION_C}°C)"
    ),
    x_label="Estimated Cell Temperature (°C)",
    y_label="Delta (W/m²)",
)
fig_abs.show()

# --- 4. Summary and Analysis ---
print("\n--- OLS Regression Results (Absolute Delta) ---")
print(ols_model_abs.summary())
print("---------------------------------------------")

# Calculate the predicted model error at the standard test condition (STC)
# cell temperature of 25°C. This value indicates the model's bias when
# the temperature correction term is zero.
TEMP_REF = 25.0
predicted_delta_at_25c_abs = ols_model_abs.predict([1, TEMP_REF])[0]
print(
    f"📈 Predicted Absolute Delta at {TEMP_REF}°C: {predicted_delta_at_25c_abs:+.2f} W/m²"
)

***

#### 7.4.2 Regression for Relative Delta

Next, we analyze the relative error as a percentage. This helps identify multiplicative biases that scale with irradiance. We filter out low-irradiance data to avoid numerical instability where small absolute errors can lead to huge relative errors.

In [ ]:
# --- Regression for Relative Delta ---

# --- 1. Data Preparation ---
# Filter out low irradiance values and low cell temperatures (using the threshold defined in 7.4.1)
df_rel_reg = df_golden_only[
    (df_golden_only["ghi_w_m2"] > 100)
    & (df_golden_only["temp_cell_estimated_c"] >= MIN_CELL_TEMP_FOR_REGRESSION_C)
].copy()

# Calculate relative delta as a percentage
df_rel_reg["delta_ghi_relative"] = (
    (df_rel_reg["ghi_modeled_w_m2"] - df_rel_reg["ghi_w_m2"])
    / df_rel_reg["ghi_w_m2"]
    * 100
)

# --- 2. OLS Regression ---
X = sm.add_constant(df_rel_reg["temp_cell_estimated_c"])
y = df_rel_reg["delta_ghi_relative"]
ols_model_rel = sm.OLS(y, X, missing="drop").fit()

# --- 3. Plotting ---
fig_rel = plot_regression_scatter(
    df_reg=df_rel_reg,
    x_col="temp_cell_estimated_c",
    y_col="delta_ghi_relative",
    ols_model=ols_model_rel,
    title=(
        f"Relative Model Error vs. Cell Temperature for {TARGET_PARK_NAME}\n"
        f"(Filtered: GHI > 100 W/m² and Cell Temp >= {MIN_CELL_TEMP_FOR_REGRESSION_C}°C)"
    ),
    x_label="Estimated Cell Temperature (°C)",
    y_label="Relative Delta (%)",
)
fig_rel.show()

# --- 4. Summary and Analysis ---
print("\n--- OLS Regression Results (Relative Delta) ---")
print(ols_model_rel.summary())
print("---------------------------------------------")

# Calculate the predicted model error at the standard test condition (STC)
# cell temperature of 25°C. This provides a direct estimate for the
# necessary adjustment to the dc_derate_factor.
TEMP_REF = 25.0
predicted_delta_at_25c_rel = ols_model_rel.predict([1, TEMP_REF])[0]
print(
    f"📈 Predicted Relative Delta at {TEMP_REF}°C: {predicted_delta_at_25c_rel:+.2f}%"
)

***
***

## 8. Final Data Export

In [ ]:
import datetime as dt

try:
    # Ensure necessary configuration variables are defined from running previous cells
    if "TARGET_PARK_NAME" not in globals() or "OUTPUT_DIR" not in globals():
        raise NameError(
            "Required configuration variables (TARGET_PARK_NAME, OUTPUT_DIR) are not defined. "
            "Please run the '1.2 Configuration' cell first."
        )

    print("Preparing data for final export...")

    # 1. Select relevant columns for the export file
    columns_to_export = [
        "ac_export_w",
        "temp_air_c",
        "wind_speed_m_s",
        "pressure_hpa",
        "is_day",
        "is_clipped",
        "is_curtailed",
        "is_shaded",
        "is_snow_covered",
        "pdc_effective_w",
        "poa_estimated_w_m2",
        "temp_cell_estimated_c",
        "dni_extra_w_m2",
        "ghi_modeled_w_m2",
        "dni_modeled_w_m2",
        "dhi_modeled_w_m2",
        "kt_modeled",
    ]
    df_export = df_30min[columns_to_export].copy()

    # 2. Invalidate data points affected by anomalies
    contamination_flags = [
        "is_clipped",
        "is_curtailed",
        "is_shaded",
        "is_snow_covered",
    ]
    columns_to_invalidate = [
        "poa_estimated_w_m2",
        "temp_cell_estimated_c",
        "ghi_modeled_w_m2",
        "dni_modeled_w_m2",
        "dhi_modeled_w_m2",
        "kt_modeled",
    ]

    contamination_mask = df_export[contamination_flags].any(axis=1)
    df_export.loc[contamination_mask, columns_to_invalidate] = np.nan
    print(f"Invalidated {contamination_mask.sum()} data points due to anomaly flags.")

    # 3. Handle nighttime values
    night_mask = ~df_export["is_day"]
    irradiance_columns = [
        "poa_estimated_w_m2",
        "ghi_modeled_w_m2",
        "dni_modeled_w_m2",
        "dhi_modeled_w_m2",
    ]
    df_export.loc[night_mask, irradiance_columns] = 0
    df_export.loc[night_mask, "temp_cell_estimated_c"] = df_export.loc[
        night_mask, "temp_air_c"
    ]
    print(f"Corrected nighttime values for {night_mask.sum()} data points.")

    # 4. Define output paths and save the CSV file
    base_filename = f"{TARGET_PARK_NAME}_irradiance_data"
    output_path_csv = OUTPUT_DIR / f"{base_filename}.csv"
    output_path_md = OUTPUT_DIR / f"{base_filename}.md"

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    df_export.to_csv(output_path_csv, index=True, index_label="timestamp_utc")

    # 5. Gather file stats and generate Markdown metadata
    csv_size_bytes = output_path_csv.stat().st_size
    if csv_size_bytes < 1024**2:
        csv_size_str = f"{csv_size_bytes / 1024:.2f} KB"
    else:
        csv_size_str = f"{csv_size_bytes / 1024**2:.2f} MB"

    metadata_md = f"""
# Metadata for {TARGET_PARK_NAME} Irradiance Data

## File Information
- **Data File:** `{output_path_csv.name}`
- **Generated On (UTC):** `{dt.datetime.now(dt.timezone.utc).isoformat()}`
- **File Size:** {csv_size_str}

## Data Coverage
- **Park Name:** `{TARGET_PARK_NAME}`
- **Time Period Start (UTC):** `{df_export.index.min()}`
- **Time Period End (UTC):** `{df_export.index.max()}`
- **Number of Rows:** `{len(df_export):,}`
- **Time Frequency:** 30 minutes

---

## Column Descriptions

- `timestamp_utc`: Timestamp in UTC for the start of the 30-minute interval.

### Input & Measured Data
- `ac_export_w`: AC power exported to the grid, in **Watts**. _Note: This is derived from hourly kWh energy by converting to average power and linearly interpolating to a 30-minute frequency._
- `temp_air_c`: Ambient air temperature, in **degrees Celsius**. _Note: Based on hourly measurements, linearly interpolated to a 30-minute frequency._
- `wind_speed_m_s`: Wind speed, in **meters per second**. _Note: Based on hourly measurements, linearly interpolated to a 30-minute frequency._
- `pressure_hpa`: Atmospheric pressure, in **hectopascals**. _Note: Based on hourly measurements, linearly interpolated to a 30-minute frequency._

### Anomaly Flags
- `is_day`: Boolean flag, `True` if the sun is above the horizon.
- `is_clipped`: Boolean flag, `True` if production is likely limited by inverter clipping.
- `is_curtailed`: Boolean flag, `True` if production is likely limited by external curtailment.
- `is_shaded`: Boolean flag, `True` if production is likely affected by inter-row shading.
- `is_snow_covered`: Boolean flag, `True` if production is likely affected by snow cover on panels.

### Modeled & Estimated Data
- `pdc_effective_w`: Estimated effective DC capacity of the plant, accounting for degradation and outages, in **Watts**.
- `poa_estimated_w_m2`: Estimated Plane of Array (POA) irradiance, derived from AC power, in **W/m²**. _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
- `temp_cell_estimated_c`: Estimated solar cell temperature, in **degrees Celsius**. _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
- `dni_extra_w_m2`: Extraterrestrial direct normal irradiance on a plane normal to the sun's rays, in **W/m²**.
- `ghi_modeled_w_m2`: Modeled Global Horizontal Irradiance (GHI), derived from estimated POA, in **W/m²**. _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
- `dni_modeled_w_m2`: Modeled Direct Normal Irradiance (DNI), derived from estimated POA, in **W/m²**. _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
- `dhi_modeled_w_m2`: Modeled Diffuse Horizontal Irradiance (DHI), derived from estimated POA, in **W/m²**. _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
- `kt_modeled`: Modeled clearness index (`ghi_modeled_w_m2` / `ghi_clearsky_w_m2`). _Note: Values are invalidated (`NaN`) during periods with active anomaly flags._
"""

    # 6. Save the metadata to a Markdown file
    with open(output_path_md, "w", encoding="utf-8") as f:
        # Use strip() to remove leading/trailing whitespace from the multiline string
        f.write(metadata_md.strip())

    print(f"✅ Successfully exported data to: {output_path_csv}")
    print(f"✅ Successfully exported metadata to: {output_path_md}")

except NameError as e:
    print(f"❌ EXPORT FAILED: {e}")
except Exception as e:
    print(f"❌ An unexpected error occurred during export: {e}")